In [103]:
# Helper function, transfer a given integer into its little endian with 7 limbs, each limb is 64 bits
def u_64_little_endian(n):
    str_hex = hex(n)
    str_hex_without_0x = str_hex[2:]
    full_width_str = '0' * (112 - len(str_hex_without_0x)) + str_hex_without_0x
    assert len(full_width_str) == 112

    res = []
    for i in range(7):
        temp = '0x' + full_width_str[112 - 16 * (i + 1) : 112 - 16 * i]
        res.append(temp)
    return res

In [104]:
p_str = "0x24000000000024000130e0000d7f70e4a803ca76f439266f443f9a5cda8a6c7be4a7a5fe8fadffd6a2a7e8c30006b9459ffffcd300000001"
p_int = int(p_str, 16)
u_64_little_endian(p_int)

['0x9ffffcd300000001',
 '0xa2a7e8c30006b945',
 '0xe4a7a5fe8fadffd6',
 '0x443f9a5cda8a6c7b',
 '0xa803ca76f439266f',
 '0x0130e0000d7f70e4',
 '0x2400000000002400']

In [105]:
# Helper function, transfer a given integer into its NAF form
# def of NAF form: https://en.wikipedia.org/wiki/Non-adjacent_form
def naf_representation(n):
    naf = []
    while n > 0:
        if n % 2 == 1:
            # If n is odd, set the current digit as 2 - (n mod 4).
            digit = 2 - (n % 4)
            n -= digit
        else:
            # If n is even, set the current digit as 0.
            digit = 0
        naf.insert(0, digit)
        n //= 2
    return naf

# Example usage:
# decimal_number = 7
# naf = naf_representation(decimal_number)
# print(naf): [1, 0, 0, -1]

## The Base Field $\mathbb{F}_p$
seed $ U = -1298074214633708060054710657220608$ \
the base field size $p = 36 U^4 + 36 U^3 + 24 U^2 + 6 U + 1 $

In [106]:
U = -1298074214633708060054710657220608
p_int = 36 * U**4 + 36 * U**3 + 24 * U**2 + 6 * U + 1
hex(p_int)

'0x24000000000024000130e0000d7f70e4a803ca76f439266f443f9a5cda8a6c7be4a7a5fe8fadffd6a2a7e8c30006b9459ffffcd300000001'

In [107]:
u_64_little_endian(p_int)

['0x9ffffcd300000001',
 '0xa2a7e8c30006b945',
 '0xe4a7a5fe8fadffd6',
 '0x443f9a5cda8a6c7b',
 '0xa803ca76f439266f',
 '0x0130e0000d7f70e4',
 '0x2400000000002400']

In [108]:
Fp = GF(p_int)
Fp

Finite Field of size 102211695604070082112571065507755096754575920209623522239390234855490679834276115250716018318118556227909439196474813090886893187366913

In [109]:
R = Fp(2**448)
R

11356855067116315761326349333718857071609919219953404605758555192204529273465116571178922486933671965396447230942486297326349317046265

### the Scalar Field $\mathbb{F}_q$
the scalar field size $q = 36 U^4 + 36 U^3 + 18 U^2 + 6 U+1$

In [110]:
q_int = 36 * U**4 + 36 * U**3 + 18 * U**2 + 6 * U + 1
hex(q_int)

'0x24000000000024000130e0000d7f70e4a803ca76f439266f443f9a5c7a8a6c7be4a775fe8e177fd69ca7e85d60050af41ffffcd300000001'

In [111]:
u_64_little_endian(q_int)

['0x1ffffcd300000001',
 '0x9ca7e85d60050af4',
 '0xe4a775fe8e177fd6',
 '0x443f9a5c7a8a6c7b',
 '0xa803ca76f439266f',
 '0x0130e0000d7f70e4',
 '0x2400000000002400']

In [112]:
Fq = GF(q_int)
Fq

Finite Field of size 102211695604070082112571065507755096754575920209623522239390234855480569854275933742834077002685857629445612735086326265689167708028929

### the field $\mathbb{F}_{p^2}$

$\mathbb{F}_{p^2} = \mathbb{F}_p[z] = \mathbb{F}_q[X] / (X^2 + 5)$ where $X^2 + 5$ is irreducible in $\mathbb{F}_p[X]$

In [113]:
Rp = PolynomialRing(Fp, "x")
x = Rp.gen()
(x^2 + 5).is_irreducible()

True

In [114]:
Fp2.<z> = Fp.extension(x^2 + 5)
Fp2

Finite Field in z of size 102211695604070082112571065507755096754575920209623522239390234855490679834276115250716018318118556227909439196474813090886893187366913^2

In [115]:
# Check the identity
z**2 + 5 == 0

True

### the field $\mathbb{F}_{p^6}$
$\mathbb{F}_{p^6} = \mathbb{F}_{p^2}[v] = \mathbb{F}_{p^2}[X] / (X^3 - 57/(z + 3))$ where $X^3 - 57/(z+3)$ is irreducible in $\mathbb{F}_{p^2}[X]$

In [116]:
I = 57 * (z + 3)**(-1)
I

21902506200872160452693799751661806447409125759205040479869336040462288535916310410867718211025404905980594113530317090904334254435763*z + 36504177001453600754489666252769677412348542932008400799782226734103814226527184018112863685042341509967656855883861818173890424059624

In [117]:
Rp2 = PolynomialRing(Fp2, "x")
x = Rp2.gen()
(x^3 - I).is_irreducible()

True

In [118]:
Fp6.<v> = Fp2.extension(x^3 - I)
Fp6

Univariate Quotient Polynomial Ring in v over Finite Field in z of size 102211695604070082112571065507755096754575920209623522239390234855490679834276115250716018318118556227909439196474813090886893187366913^2 with modulus v^3 + 80309189403197921659877265756093290307166794450418481759520898815028391298359804839848300107093151321928845082944495999982558932931150*z + 65707518602616481358081399254985419342227377277615121439608008121386865607748931232603154633076214717941782340590951272713002763307289

In [119]:
Fp6.is_field()

True

In [120]:
# Check the identity
v**3 - I == 0

True

### The const FROBENIUS_COEFF_FQ6_C1[1] 

In [121]:
I**((p_int-1)/3)

92529011805995300781026747858635174615077510851648588563707597606861718583198168205651139531970571313972517572887035428707503415341858*z + 51260142370505185497351973260211965617805492386872277241342529142140236670911574395535108090223527812381417682871589681672697256676611

In [122]:
# c0
u_64_little_endian(51260142370505185497351973260211965617805492386872277241342529142140236670911574395535108090223527812381417682871589681672697256676611)

['0xa001825382850d03',
 '0x52e787634eccc335',
 '0x041bc4bb14cc42dc',
 '0x6f93678bc921dcd5',
 '0xe3886170077a5098',
 '0x3bc0d351f4c70da1',
 '0x120de97f024c55bc']

In [123]:
# c1
u_64_little_endian(92529011805995300781026747858635174615077510851648588563707597606861718583198168205651139531970571313972517572887035428707503415341858)

['0x2772fea378a9b322',
 '0x13be1b7cd3c9ea0e',
 '0x5efa1ddf46580804',
 '0xa1a7f5502b709196',
 '0x132461908eadbe3d',
 '0xd82becc2ef081b76',
 '0x2096f3f804d973af']

### The const FROBENIUS_COEFF_FQ6_C1[2] 

In [124]:
I**((p_int**2-1)/3)
# c1 = 0

39370513046094319542878173447389497729725081225711316008956793976697167703016365005507455943322894334

In [125]:
# c0
u_64_little_endian(39370513046094319542878173447389497729725081225711316008956793976697167703016365005507455943322894334)

['0x100004c37ffffffe',
 '0xc8ad8b38dffaf50c',
 '0xc956d01c903d720d',
 '0x50000d7ee0e4a803',
 '0x00000000360001c9',
 '0x0000000000004800',
 '0x0000000000000000']

### The const XI_TO_Q_MINUS_1_OVER_2

In [126]:
I**((p_int-1)/2)

65079093581113076137070936836706683307023886770993453415929204320012344240499906559031972253911619563827341548210931723067336455786996*z + 95958235239618370663357236287665721330778695800924622118898462510960785934385719569712082879717746516106642479264739976661558948422166

In [127]:
# c0
u_64_little_endian(95958235239618370663357236287665721330778695800924622118898462510960785934385719569712082879717746516106642479264739976661558948422166)

['0x54cf5ad1c0926216',
 '0x186c1f3ce4a46d4e',
 '0x9c23800ce9c9452f',
 '0x50e0d09ff6d6c08b',
 '0x7cf421e4d46f6666',
 '0x678664ba4b6d8343',
 '0x21cc26d5de0f80f4']

In [128]:
# c1
u_64_little_endian(65079093581113076137070936836706683307023886770993453415929204320012344240499906559031972253911619563827341548210931723067336455786996)

['0xc0505f4c260e91f4',
 '0xe7bbd15f10723657',
 '0xb4b3e0c35358097e',
 '0x87c56f42a558750d',
 '0x4b7211d23f34f0ae',
 '0xf6839d29e2f0d250',
 '0x16ebe8b2e12a1106']

### The const NEG_SIX_U_PLUS_2_NAF

In [129]:
L = naf_representation(-(6*U + 2))
L[::-1]

[0,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 -1,
 0,
 -1,
 0,
 1,
 0,
 -1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 1]